In [1]:
import pandas as pd
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

In [4]:
# Carrega o dataset a partir de um arquivo CSV
dataset = pd.read_csv("framingham_heart_disease.csv")

dataset.head()


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
# Remove as linhas com valores faltantes (NaN) do dataset
dataset = dataset.dropna()

# Seleciona todas as colunas, exceto a última, e converte em um array NumPy (features/atributos)
X = dataset.iloc[:, :-1].values

# Seleciona a última coluna do dataset e converte em um array NumPy (target/variável de saída)
y = dataset.iloc[:, -1].values

In [6]:
# Separar dados em Treino e Teste
 
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size = 1/5, random_state = 0)

In [7]:
# Treinando o modelo
 
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
 

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [8]:
# Prever Valor Especifico
 
print(classifier.predict([[1,30,4,0,0,0,0,0,0,195,130,70,80,20,56]]))

[0]


In [11]:
# Exemplo de dados de teste (X_test)

# Cada linha representa um paciente, e as colunas são características (idade, pressão arterial, colesterol, etc.)
#X_test = [
#    [45, 130, 85],  # Paciente 1
#    [50, 140, 90],  # Paciente 2
#    [60, 150, 100]  # Paciente 3
#]

###########################################################################################################################################

# Realiza a previsão das classes (rótulos) para os dados de teste
# O modelo vai prever 0 (não tem a doença) ou 1 (tem a doença) para cada paciente
y_pred = classifier.predict(X_test) # Resultado: [0, 1, 0]



# Realiza a previsão das probabilidades para cada classe nos dados de teste
# predict_proba retorna a probabilidade de ser classe 0 (primeira coluna) e a probabilidade de ser classe 1 (segunda coluna)
y_pred_prob = classifier.predict_proba(X_test)
# Resultado: [[0.85, 0.15],  # Paciente 1 (85% de chance de NÃO ter a doença, 15% de chance de ter)
#             [0.30, 0.70],  # Paciente 2 (30% de chance de NÃO ter, 70% de chance de ter)
#             [0.95, 0.05]]  # Paciente 3 (95% de chance de NÃO ter, 5% de chance de ter)





# Seleciona apenas as probabilidades da classe positiva (coluna 1), 
# ou seja, a probabilidade de o evento ocorrer (como a presença de doença)
y_pred_prob = y_pred_prob[:, 1]

# Agora vamos combinar as previsões com as probabilidades
# Paciente 1: Previsão 0 (não tem a doença), Probabilidade de 15% de ter a doença
# Paciente 2: Previsão 1 (tem a doença), Probabilidade de 70% de ter a doença
# Paciente 3: Previsão 0 (não tem a doença), Probabilidade de 5% de ter a doença
y_result_prob = np.concatenate((y_pred.reshape(len(y_pred), 1), y_pred_prob.reshape(len(y_pred_prob), 1)), axis=1)
# Resultado:
# [[0.   0.15]   Paciente 1: Não tem a doença, 15% de chance de ter
#  [1.   0.70]   Paciente 2: Tem a doença, 70% de chance de ter
#  [0.   0.05]]  Paciente 3: Não tem a doença, 5% de chance de ter


In [12]:
# Importa a função confusion_matrix para gerar a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Exibe a matriz de confusão
# A matriz de confusão mostra a comparação entre as previsões do modelo (y_pred)
# e os valores reais (y_test):
# [ [TN, FP], 
#   [FN, TP] ]
# TN (True Negatives): o modelo previu 0 corretamente
# FP (False Positives): o modelo previu 1 incorretamente (falsos positivos)
# FN (False Negatives): o modelo previu 0 incorretamente (falsos negativos)
# TP (True Positives): o modelo previu 1 corretamente
print(cm)

# Concatena as previsões do modelo (y_pred) com os valores reais (y_test) para comparação
# O resultado é uma matriz com duas colunas:
# - Primeira coluna: as previsões feitas pelo modelo
# - Segunda coluna: os valores reais do conjunto de teste
# Assim, cada linha dessa matriz mostra um par (previsão, valor real)
y_result = np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), axis=1)

# Exibe a matriz com previsões e valores reais para facilitar a visualização
print(y_result)

# Calcula a acurácia do modelo, que é a proporção de previsões corretas (acertos)
# em relação ao total de previsões feitas
# Fórmula da acurácia: (TP + TN) / (Total de amostras)
print(accuracy_score(y_test, y_pred))


[[620   3]
 [103   6]]
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
0.855191256830601
